## Setup

### Imports

In [3]:
import torch
import numpy as np
import pandas as pd
from itertools import product
from src.forward import *
from src.knn import *
from src.ftm import constants as FTM_constants
import IPython.display as ipd

device = "cuda" if torch.cuda.is_available() else "cpu"

### Phi definition

In [ ]:
class FIRFilter(torch.nn.Module):
    
    def __init__(self, filter_type="hp", coef=0.85, fs=44100, ntaps=101, plot=False):

        """Initilize FIR pre-emphasis filtering module."""
        super(FIRFilter, self).__init__()
        self.filter_type = filter_type
        self.coef = coef
        self.fs = fs
        self.ntaps = ntaps
        self.plot = plot

        import scipy.signal

        if ntaps % 2 == 0:
            raise ValueError(f"ntaps must be odd (ntaps={ntaps}).")

        if filter_type == "hp":
            self.fir = torch.nn.Conv1d(1, 1, kernel_size=3, bias=False, padding=1)
            self.fir.weight.requires_grad = False
            self.fir.weight.data = torch.tensor([1, -coef, 0]).view(1, 1, -1)
        elif filter_type == "fd":
            self.fir = torch.nn.Conv1d(1, 1, kernel_size=3, bias=False, padding=1)
            self.fir.weight.requires_grad = False
            self.fir.weight.data = torch.tensor([1, 0, -coef]).view(1, 1, -1)
        elif filter_type == "aw":
            # Definition of analog A-weighting filter according to IEC/CD 1672.
            f1 = 20.598997
            f2 = 107.65265
            f3 = 737.86223
            f4 = 12194.217
            A1000 = 1.9997

            NUMs = [(2 * np.pi * f4) ** 2 * (10 ** (A1000 / 20)), 0, 0, 0, 0]
            DENs = np.polymul(
                [1, 4 * np.pi * f4, (2 * np.pi * f4) ** 2],
                [1, 4 * np.pi * f1, (2 * np.pi * f1) ** 2],
            )
            DENs = np.polymul(
                np.polymul(DENs, [1, 2 * np.pi * f3]), [1, 2 * np.pi * f2]
            )

            # convert analog filter to digital filter
            b, a = scipy.signal.bilinear(NUMs, DENs, fs=fs)

            # compute the digital filter frequency response
            w_iir, h_iir = scipy.signal.freqz(b, a, worN=512, fs=fs)

            # then we fit to 101 tap FIR filter with least squares
            taps = scipy.signal.firls(ntaps, w_iir, abs(h_iir), fs=fs)

            # now implement this digital FIR filter as a Conv1d layer
            self.fir = torch.nn.Conv1d(
                1, 1, kernel_size=ntaps, bias=False, padding=ntaps // 2
            )
            self.fir.weight.requires_grad = False
            self.fir.weight.data = torch.tensor(taps.astype("float32")).view(1, 1, -1)

        self.fir.weight.data = self.fir.weight.data.to(device)

    def forward(self, input):
        """Calculate forward propagation.
        Args:
            input (Tensor): Predicted signal (B, #channels, #samples).
            target (Tensor): Groundtruth signal (B, #channels, #samples).
        Returns:
            Tensor: Filtered signal.
        """
        input = torch.nn.functional.conv1d(
            input.unsqueeze(0).to(torch.float), self.fir.weight.data, padding=self.ntaps // 2
        )
        return input.squeeze(0).to(torch.float)

### Naive k neighbours search

In [ ]:
def dist_naive_factory(theta_ref,phi,logscale):
    #calculation of the audio for the reference node
    audio_ref = rectangular_drum(theta_ref, logscale=logscale,**FTM_constants)
    phi_ref = phi(audio_ref)
    def naiveDistFunction(theta):
        audio_node = rectangular_drum(theta, logscale=logscale,**FTM_constants)
        phi_node = phi(audio_node)
        return (torch.sum(torch.pow(torch.subtract(phi_ref, phi_node), 2), dim=0)).cpu().detach().numpy()
    return naiveDistFunction

### Approximated k neighbours search

In [ ]:
# First we need to create the M matrix

# M(theta0) = grad(Phi o g)(theta0).T * grad(Phi o g)(theta0)
# This return M = f(theta0)

def M_from_G(G):
    return torch.matmul(torch.transpose(G,0,1),G)

def M_from_theta(theta, G):
    return M_from_G(G(inputs=theta))

def M_factory(logscale,Phi):
    S_from_theta = pknn_forward_factory(logscale,Phi)
    #G = torch.func.jacfwd(S_from_theta)
    G = functools.partial(torch.autograd.functional.jacobian, func=S_from_theta, create_graph=False,strategy="forward-mode",vectorize=True)
    M = functools.partial(M_from_theta,G=G)
    return M

def dist_from_M_and_theta0(t_candidat, t_ref, M):
    return np.matmul(np.matmul(np.transpose(t_ref-t_candidat),M.cpu().detach().numpy()),t_ref-t_candidat)

def dist_approximated_factory(t_ref,Phi,logscale):
    M = M_factory(logscale,Phi)
    M = M(torch.tensor(t_ref, requires_grad=True).to(device))
    return functools.partial(dist_from_M_and_theta0, M=M, t_ref=t_ref)

### Create Parameters Dataset

In [ ]:
# Create DataFrame and write it to a CSV file for later use

def create_DF(bounds, subdiv, path):
    
    #Linspace of every parameters of size k
    Dbase = np.zeros((subdiv,5))
    for i in range(5):
        Dbase[:,i] = np.linspace(bounds[1][i][0],bounds[1][i][1],subdiv)
    baseDF = pd.DataFrame(data=Dbase,columns=bounds[0])

    #Product of the linspaces to get all the possible combinations (size subdiv**5, will take time)
    D = list(product(baseDF['omega'],baseDF['tau'],baseDF['p'],baseDF['d'],baseDF['alpha']))
    DF = pd.DataFrame(data=D,columns=bounds[0])

    DF.to_csv(path)
    
    return DF


## Main

In [ ]:
# Boundaries

bounds = [['omega', 'tau', 'p', 'd', 'alpha'],
 [(2.400247964468862, 3.798136579655672),
  (0.0700188044714488, 0.7999966616122908),
  (-4.999978530884291, -0.6989804486272966),
  (-4.99983759075039, -0.5229983775344527),
  (1.2362882382361523e-05, 0.9999649724709304)]]

# Only run this to recreate the parameters CSV, this can take a long time to finish depending on the subdivision

create_DF(bounds=bounds, subdiv=5, path='data/default_parameters.csv')


,omega,tau,p,d,alpha
0,2.400248,0.070019,-4.999979,-4.999838,0.000012
1,2.400248,0.070019,-4.999979,-4.999838,0.250001
2,2.400248,0.070019,-4.999979,-4.999838,0.499989
3,2.400248,0.070019,-4.999979,-4.999838,0.749977
4,2.400248,0.070019,-4.999979,-4.999838,0.999965
...,...,...,...,...,...
3120,3.798137,0.799997,-0.698980,-0.522998,0.000012
3121,3.798137,0.799997,-0.698980,-0.522998,0.250001
3122,3.798137,0.799997,-0.698980,-0.522998,0.499989
3123,3.798137,0.799997,-0.698980,-0.522998,0.749977


In [ ]:
DatasetPath = "data/default_parameters.csv"
parameters_name = ["omega","tau","p","d","alpha"]
logscale = True
Phi = FIRFilter()

k = 5 
theta_ref_index = 0

#Get the reference Theta
print("Reading Dataset")

DS = pd.read_csv(DatasetPath, index_col=0)
theta_ref_line = DS.iloc[[theta_ref_index]]
theta_ref = [ theta_ref_line[parameters_name[k]].iloc[0] for k in range(len(parameters_name)) ]

#Find the knn (Naive)
print("Naive knn")

dist = dist_naive_factory(theta_ref, Phi, logscale)
naive_neighbours,naive_time = find_neighbour(DatasetPath,k,dist,return_time=True,show_progress=True)

#Find the knn (Approx)
print("Approx knn")

dist = dist_approximated_factory(theta_ref, Phi, logscale)
approx_neighbours,approx_time = find_neighbour(DatasetPath,k,dist,return_time=True,show_progress=True)


In [ ]:
#Write the neighbours to a CSV file 

print('naive:')
naive_DL = np.transpose(np.column_stack((np.array(theta_ref),np.transpose(np.array(naive_neighbours)))))
naive_DF = pd.DataFrame(naive_DL, columns=(parameters_name))
print(naive_DF)
naive_DF.to_csv("data/naive_knn.csv")

print('\napprox:')
approx_DL = np.transpose(np.column_stack((np.array(theta_ref),np.transpose(np.array(approx_neighbours)))))
approx_DF = pd.DataFrame(approx_DL, columns=(parameters_name))
print(approx_DF)
approx_DF.to_csv("data/approx_knn.csv")

# Method characterization

In [61]:
#Defined the list of parameters
approx_neighbours = []
naive_neighbours = []

parameters_name = ['omega', 'tau', 'p', 'd', 'alpha']

#Recover all parameters
data = pd.read_csv("data/naive_knn.csv", index_col=0)
data_size = data.size
for i in range(int(data_size/5)):
    #Get phi of the neighbour
    parameterLine = data.iloc[[i]]
    theta = np.array([ parameterLine[parameters_name[k]].iloc[0] for k in range(len(parameters_name)) ])
    approx_neighbours.append(theta)

#Recover all parameters
data1 = pd.read_csv("data/approx_knn.csv", index_col=0)
data_size = data.size
for i in range(int(data_size/5)):
    #Get phi of the neighbour
    parameterLine = data1.iloc[[i]]
    theta = np.array([ parameterLine[parameters_name[k]].iloc[0] for k in range(len(parameters_name)) ])
    naive_neighbours.append(theta)

In [63]:
#LISTEN HERE the truth

#Calulate audios from the parameters
for i in range(len(naive_neighbours)):
    print("Neighbours n°",i+1)
    audio = rectangular_drum(naive_neighbours[i],logscale=True,**FTM_constants)
    ipd.display(ipd.Audio(data=audio, rate=44100))



Neighbours n° 1


Neighbours n° 2


Neighbours n° 3


Neighbours n° 4


Neighbours n° 5


Neighbours n° 6


In [64]:
#LISTEN HERE the approximation

#Calulate audios from the parameters
approx_neighbours_audios = []
for i in range(len(approx_neighbours)):
    print("Neighbours n°",i+1)
    audio = rectangular_drum(naive_neighbours[i], True,**FTM_constants)
    ipd.display(ipd.Audio(data=audio, rate=44100))


Neighbours n° 1


Neighbours n° 2


Neighbours n° 3


Neighbours n° 4


Neighbours n° 5


Neighbours n° 6


In [65]:
#Precision and recall

t_p = 0
f_p = 0

def inList(element,list_l):
    for i in range(len(list_l)):
        equal = True
        for k in range(len(list_l[i])):
            if(list_l[i][k]!= element[k]):
                equal = False
        if (equal):
            return True

for i in range(len(naive_neighbours)):
    if (inList(approx_neighbours[i],naive_neighbours)):
        t_p += 1
    else:
        f_p += 1
print("Precision : ",t_p/len(naive_neighbours))
print("Recall : ",f_p/len(naive_neighbours))

Precision :  0.8333333333333334
Recall :  0.16666666666666666
